In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/phase-2-sentiment-analysis/df_cleaned1.csv
/kaggle/input/phase-3-wordtokenioze/df_new.csv


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from collections import Counter
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
from sklearn.feature_extraction.text import CountVectorizer

import re
from os import path
from PIL import Image
import plotly

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [5]:
df = pd.read_csv('/kaggle/input/phase-3-wordtokenioze/df_new.csv')
df.head(3)

,Unnamed: 0,date_review,location,job_title,tenure,Current Employee,headline,pros,cons,recommend,...,diversity_inclusion,career_opp,comp_benefits,senior_mgmt,s_polarity,s_subjectivity,p_polarity,p_subjectivity,c_polarity,c_subjectivity
0,0,2008-01-31,"Cupertino, CA",Senior Product Manager,NaN,1.0,We make products that poeple enjoy buying and ...,The people we work with are great and I can't ...,You have to be careful because this job can ta...,1,...,NaN,3.0,4.0,4.0,0.400000,0.500000,0.8000,0.750000,-0.016667,1.000000
1,1,2008-02-05,"Mountain View, CA",Software Engineer,NaN,1.0,Freedom within the company and lots of things ...,I work w/ a diverse set of people. You have ha...,Google can be a bit overwhelming w/ the volume...,1,...,NaN,4.0,4.0,4.0,0.000000,0.000000,0.8000,0.750000,0.233333,0.441667
2,2,2008-02-11,"Mountain View, CA",Software Engineer,NaN,1.0,Work with smart people who are willing to shar...,People are willing to share knowledge which is...,A company like this requires a lot of long hou...,1,...,NaN,5.0,5.0,5.0,0.232143,0.696429,0.4125,0.620833,-0.170833,0.470833


In [6]:
def tokenizer(col , num):
    line = df[col].to_string()
    tokens = [w for w in word_tokenize(line.lower()) if w.isalpha()]
    no_stop = [n for n in tokens if n not in stopwords.words('english')]
    count = Counter(no_stop).most_common(num)
    
    return count
    
    

In [ ]:
top20_headline = tokenizer('headline' , 20)
top20_headline

In [ ]:
top20_pros = tokenizer('pros',20)
pd.DataFrame(top20_pros , columns = ['word','count']).T

In [ ]:
top20_cons = tokenizer('cons', 20)
pd.DataFrame(top20_cons  , columns = ['word','count']).T

In [ ]:
stop_words = set(stopwords.words('english'))
custom = ["work" , "place" ,"they" , "there"]
stop_words = stop_words.union(custom)

**Visualizing the most common Unigram Words**

In [ ]:
def plot_wordcloud(col):
    txt = df[col].to_string()
    wordcloud = WordCloud(background_color = 'white',
                         stopwords = stop_words,
                         max_words = 100,
                         max_font_size = 50,
                         random_state = 12).generate(txt)
    plt.figure(figsize = (10,10))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.title(col,fontsize = 15)
    plt.show()

In [ ]:
for col in ['headline','pros','cons']:
    plot_wordcloud(col)

**Findings:**

* The most common words in Summary are positive, which is consistent with our previous plot where 60% of employees recommend this company.
* Employees love this company because of its great benefits , good people and good work-life balance.
* It appears employees' complaints are mostly about worklife balance , long hour and the job nature of call centers.

**Visualize The Most Common Bi-grams Words**

In [7]:
def word_lemmatize(txt):
    
    lem = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))
    if not pd.isna(txt):
        token = [t for t in word_tokenize(txt.lower()) if t.isalpha()]
        no_stop = [w for w in token if w not in stop_words]
        return " ".join(lem.lemmatize(w) for w in no_stop)
    else:
        return np.NaN
    

In [8]:
df["headline_corpus"] = df["headline"].apply(word_lemmatize)
df["pros_corpus"] = df["pros"].apply(word_lemmatize)
df["cons_corpus"] = df["cons"].apply(word_lemmatize)

In [ ]:
df[['headline_corpus' , 'pros_corpus' , 'cons_corpus']].head()

In [ ]:
def get_top_bigram(corpus,n):
    vec =  CountVectorizer(ngram_range = (2,2)).fit(corpus)
    tra = vec.transform(corpus)
    sum_words = tra.sum(axis = 0) #sum the whole column and produces a matrix of [0][col.number]
    
    freq = [(w,sum_words[0,i]) for w,i in vec.vocabulary_.items()]
    # vec.vocabulary gives a dict of keys with its position
    # ,,.items() givecs array of tuples 
    # alternative : freq = [(w,sum_words[0,vec.vocabulary_[]]) for w in vec.vocabulary_]
    freq = sorted(freq , key = lambda x:x[1] , reverse = True)
    
    return freq[:n]

In [ ]:
txt = ["make product poeple enjoy buying using" , "freedom within company lot thing learn",
        "work smart people willing share information" , "work smart people willing share information",
      "different job different creative product work"]

vec = CountVectorizer(ngram_range=(2,2)).fit(txt)
tran = vec.transform(txt)
sum_w = tran.sum(axis = 0)
print(vec.get_feature_names_out())
print(tran.toarray())
print(sum_w)
print(vec.vocabulary_.items())

In [ ]:
for i in vec.vocabulary_:
    print(i,"->",vec.vocabulary_[i])

In [ ]:
def plot_top_bigram(data , title):
    
    sns.set(rc = {'figure.figsize' : (10,8)})
    h = sns.barplot(x = 'bigram' , y = 'frequency' , data = data)
    h.set_xticklabels(h.get_xticklabels() , rotation = 90)
    plt.title(title , fontsize = 20)
    plt.show()
    

In [ ]:
headline_sum = get_top_bigram(df['headline_corpus'].dropna(),20)
headline_sum_df = pd.DataFrame(headline_sum , columns = ['bigram','frequency'])

plot_top_bigram(headline_sum_df , "Top 20 Bi-gram words in Headline")

In [ ]:
pros_sum = get_top_bigram(df['pros_corpus'].dropna(),20)
pros_sum_df = pd.DataFrame(pros_sum , columns = ['bigram','frequency'])

plot_top_bigram(pros_sum_df , "Top 20 Bi-gram words in pros")

In [ ]:
cons_sum = get_top_bigram(df['cons_corpus'].dropna(),20)
cons_sum_df = pd.DataFrame(cons_sum , columns = ['bigram','frequency'])

plot_top_bigram(cons_sum_df , "Top 20 Bi-gram words in cons")

**Creat Corpus**

In [9]:
headline_corpus_list = df['headline_corpus'].dropna().str.split()
pros_corpus_list = df['pros_corpus'].dropna().str.split()
cons_corpus_list = df['cons_corpus'].dropna().str.split()

In [10]:
headline_corpus_list.head()

0        [make, product, poeple, enjoy, buying, using]
1        [freedom, within, company, lot, thing, learn]
2    [work, smart, people, willing, share, informat...
3    [different, job, different, creative, product,...
4          [one, best, job, life, work, great, people]
Name: headline_corpus, dtype: object

**Bag Of Words**

In [11]:
import gensim
from gensim.corpora import Dictionary as dict
from gensim import corpora,models
from gensim.models import CoherenceModel

In [12]:
head_dict = dict(headline_corpus_list)
pros_dict = dict(pros_corpus_list)
cons_dict = dict(cons_corpus_list)

In [ ]:
for i, (j,k) in enumerate(head_dict.iteritems()):
    print(j ," " , k)
    if i>10:
        break

In [13]:
head_bow = [head_dict.doc2bow(line) for line in headline_corpus_list]
pros_bow = [pros_dict.doc2bow(line) for line in pros_corpus_list]
cons_bow = [cons_dict.doc2bow(line) for line in cons_corpus_list]

In [14]:
head_bow[0]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]

**TF-IDF**

In [15]:
headline_tfidf = models.TfidfModel(head_bow)[head_bow]
pros_tfidf = models.TfidfModel(pros_bow)[pros_bow]
cons_tfidf = models.TfidfModel(cons_bow)[cons_bow]

In [ ]:
for i,j in pros_tfidf[0]:
    print(head_dict[i], "->" , j)

In [16]:
import joblib

In [17]:
def run_lda(corpus , dictionary , n_passes , n_topics , file_name):
    lda = models.LdaMulticore(corpus = corpus , passes = n_passes , num_topics = n_topics ,
                              id2word =  dictionary , random_state = 20)
    
    joblib.dump(lda.print_topics() , f"{file_name}_topics")
    joblib.dump(lda , f"{file_name}_lda")

**Topic Modeling With LDA**

**Bag Of Words**

In [ ]:
run_lda(head_bow , head_dict , 50 , 5 , "head_bow")

In [ ]:
head_bow_topics = joblib.load("head_bow_topics")
head_bow_topics

In [ ]:
run_lda(pros_bow , pros_dict , 50 , 5 , "pros_bow")

In [ ]:
pros_bow_topics = joblib.load("pros_bow_topics")
pros_bow_topics

In [ ]:
run_lda(cons_bow , cons_dict , 50 , 5 , "cons_bow")

In [ ]:
cons_bow_topics = joblib.load("cons_bow_topics")
cons_bow_topics

**TF-IDF**

In [ ]:
run_lda(headline_tfidf , head_dict , 50 , 5 , "head_tfidf")

In [ ]:
headline_tfidf_topics = joblib.load("head_bow_topics")
headline_tfidf_topics

In [ ]:
run_lda(pros_tfidf , pros_dict , 50 , 5 , "pros_tfidf")

In [ ]:
pros_tfidf_topics = joblib.load("pros_tfidf_topics")
pros_tfidf_topics

In [ ]:
run_lda(cons_tfidf , cons_dict , 50 , 5 , "cons_tfidf")

In [ ]:
cons_tfidf_topics = joblib.load("cons_tfidf_topics")
cons_tfidf_topics

**Model Evaluation**

In [ ]:
pros_bow_lda = joblib.load("pros_bow_lda")
pros_tfidf_lda = joblib.load("pros_tfidf_lda")

In [ ]:
pros_test = [x.split() for x in df["pros_corpus"].to_list()]

In [ ]:
coherence_model_bow = CoherenceModel(model = pros_bow_lda , texts = pros_test ,
                               dictionary  = pros_dict , coherence = 'c_v')

coherence_bow = coherence_model_bow.get_coherence()

print("Coherence scoree :  " , coherence_bow)

In [ ]:
coherence_model_tfidf = CoherenceModel(model = pros_tfidf_lda , texts = pros_test ,
                               dictionary  = pros_dict , coherence = 'c_v')

coherence_tfidf = coherence_model_tfidf.get_coherence()

print("Coherence scoree :  " , coherence_tfidf)

In [ ]:
df.loc[1025 , 'pros']

In [ ]:
for i , score in sorted(pros_bow_lda[pros_bow[1025]]  ,key = lambda x: x[1]  ,reverse = True):
    print(f"{i} , {score} , {pros_bow_lda.print_topic(i,5)}")


In [ ]:
for i , score in sorted(pros_tfidf_lda[pros_bow[1025]]  ,key = lambda x: x[1]  ,reverse = True):
    print(f"{i} , {score} , {pros_tfidf_lda.print_topic(i,5)}")

**Test Model on unseen data**

In [ ]:
unseen_text = "the pay is good and the work life balance is also good "
tran_text  = word_lemmatize(unseen_text)

In [ ]:
bow_text = pros_dict.doc2bow(tran_text.split())
tfidf_text = models.TfidfModel(pros_bow)[bow_text]

In [ ]:
for i , score in sorted(pros_bow_lda[bow_text]  ,key = lambda x: x[1]  ,reverse = True):
    print(f"{i} , {score} , {pros_bow_lda.print_topic(i,5)}")

In [ ]:
for i , score in sorted(pros_tfidf_lda[tfidf_text]  ,key = lambda x: x[1]  ,reverse = True):
    print(f"{i} , {score} , {pros_tfidf_lda.print_topic(i,5)}")

**Analyze unhappy group with spacy**

In [6]:

unhappy_cre_mem = df.loc[df['job_title']==' Crew Member',['headline','pros','cons']]
unhappy_ano_eme = df.loc[df['job_title']==' Anonymous Employee',['headline','pros','cons']]
unhappy_un_title = df.loc[df['job_title']==' unknown title',['headline','pros','cons']]

In [7]:
len(unhappy_cre_mem),len(unhappy_ano_eme),len(unhappy_un_title)

(7716, 155836, 75719)

In [8]:
import spacy
from spacy.matcher import Matcher

def find_adj_noun(txt):
    
    txt = txt.lower()
    nlp = spacy.load('en_core_web_sm')

    match = Matcher(nlp.vocab)
    pattern = [{'POS': 'ADJ'}, {'POS': 'NOUN'},{'POS': 'NOUN', 'OP': '?'}]
    
    match.add('pattern_name',[pattern])
    doc = nlp(txt)
    matches = match(doc)
    
    matched_list = []
    for match_id, start, end in matches:
        matched_span = doc[start:end]
        matched_list.append(matched_span.text)
        
    return matched_list

In [9]:
con_matched_cremem = find_adj_noun(txt = unhappy_cre_mem ['cons'].to_string())
Counter(con_matched_cremem).most_common(20)

[('low pay', 224),
 ('rude customers', 209),
 ('long hours', 154),
 ('bad management', 100),
 ('hard work', 82),
 ('minimum wage', 81),
 ('poor management', 70),
 ('enough hours', 52),
 ('fast food', 51),
 ('bad pay', 48),
 ('long shifts', 45),
 ('poor pay', 34),
 ('much work', 29),
 ('little pay', 29),
 ('low wage', 28),
 ('rude managers', 28),
 ('low wages', 26),
 ('terrible management', 24),
 ('horrible management', 22),
 ('fast pace', 21)]

In [14]:
unhappy_ano_eme_1 = unhappy_ano_eme.sample(n = 15000 , random_state = 20)
con_matched_anoeme = find_adj_noun(txt = unhappy_ano_eme_1['cons'].to_string())
Counter(con_matched_anoeme).most_common(20)

[('long hours', 468),
 ('low pay', 151),
 ('poor management', 74),
 ('busy season', 70),
 ('senior management', 70),
 ('low salary', 68),
 ('big company', 52),
 ('large company', 51),
 ('bad management', 50),
 ('hard work', 43),
 ('poor work', 40),
 ('upper management', 40),
 ('red tape', 35),
 ('much work', 34),
 ('slow growth', 32),
 ('internal politics', 31),
 ('high pressure', 31),
 ('long work', 28),
 ('much politics', 27),
 ('poor pay', 25)]

In [17]:
unhappy_un_title_1 = unhappy_un_title.sample(n = 15000 , random_state = 20)
con_matched_untitle = find_adj_noun(txt = unhappy_un_title_1['cons'].to_string())
Counter(con_matched_untitle).most_common(20)

[('long hours', 595),
 ('low pay', 152),
 ('busy season', 83),
 ('low salary', 74),
 ('big company', 67),
 ('senior management', 66),
 ('hard work', 60),
 ('poor management', 60),
 ('upper management', 59),
 ('large company', 51),
 ('poor work', 44),
 ('much work', 43),
 ('red tape', 37),
 ('bad management', 37),
 ('high pressure', 34),
 ('rude customers', 31),
 ('many people', 31),
 ('great place', 29),
 ('long work', 29),
 ('low salaries', 27)]